### Homework 5: Question search engine

Remeber week01 where you used GloVe embeddings to find related questions? That was.. cute, but far from state of the art. It's time to really solve this task using context-aware embeddings.

__Warning:__ this task assumes you have seen `seminar.ipynb`!

In [1]:
%pip install --upgrade transformers datasets accelerate deepspeed
import torch
import torch.nn as nn
import torch.nn.functional as F
import transformers
import datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 37.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 511.6/511.6 kB 26.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 22.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.0/54.0 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 180.7/180.7 kB 12.1 MB/s eta 0:00:00
  Created wheel for deepspeed: filename=deepspeed-0.18.2-py3-none-any.whl size=1763310 sha256=646899832d5391393a575e100c78c19f93c8ebd1f090fc4397d83f12f06f4cdf
  Stored in directory: /root/.cache/pip/wheels/69/ad/2e/e03d4739ddc0417efd8a120c2b9e784005aa226037e558c163
Successfully built deepspeed
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 18.1.0
    Uninstalling pyarrow-18.1.0:
      Successfully uninstalled pyarrow-18.1.0
  Attempting uninstall: datasets
    Found existing installation: datasets 4.0.0
    Uninstalling datasets-

### Load data and model

In [2]:
qqp = datasets.load_dataset('SetFit/qqp')
print('\n')
print("Sample[0]:", qqp['train'][0])
print("Sample[3]:", qqp['train'][3])

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/313 [00:00<?, ?B/s]

Repo card metadata block was not found. Setting CardData to empty.


train.jsonl:   0%|          | 0.00/70.8M [00:00<?, ?B/s]

validation.jsonl: 0.00B [00:00, ?B/s]

test.jsonl:   0%|          | 0.00/76.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/363846 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/40430 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/390965 [00:00<?, ? examples/s]



Sample[0]: {'text1': 'How is the life of a math student? Could you describe your own experiences?', 'text2': 'Which level of prepration is enough for the exam jlpt5?', 'label': 0, 'idx': 0, 'label_text': 'not duplicate'}
Sample[3]: {'text1': 'What can one do after MBBS?', 'text2': 'What do i do after my MBBS ?', 'label': 1, 'idx': 3, 'label_text': 'duplicate'}


In [3]:
model_name = "gchhablani/bert-base-cased-finetuned-qqp"
tokenizer = transformers.AutoTokenizer.from_pretrained(model_name)
model = transformers.AutoModelForSequenceClassification.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/320 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/890 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/433M [00:00<?, ?B/s]

### Tokenize the data

In [4]:
MAX_LENGTH = 128
def preprocess_function(examples):
    result = tokenizer(
        examples['text1'], examples['text2'],
        padding='max_length', max_length=MAX_LENGTH, truncation=True
    )
    result['label'] = examples['label']
    return result

qqp_preprocessed = qqp.map(preprocess_function, batched=True)

Map:   0%|          | 0/363846 [00:00<?, ? examples/s]

Map:   0%|          | 0/40430 [00:00<?, ? examples/s]

Map:   0%|          | 0/390965 [00:00<?, ? examples/s]

In [5]:
print(repr(qqp_preprocessed['train'][0]['input_ids'])[:100], "...")

[101, 1731, 1110, 1103, 1297, 1104, 170, 12523, 2377, 136, 7426, 1128, 5594, 1240, 1319, 5758, 136,  ...


### Task 1: evaluation (1 point)

We randomly chose a model trained on QQP - but is it any good?

One way to measure this is with validation accuracy - which is what you will implement next.

Here's the interface to help you do that:

In [6]:
val_set = qqp_preprocessed['validation']
val_loader = torch.utils.data.DataLoader(
    val_set, batch_size=1, shuffle=False, collate_fn=transformers.default_data_collator
)

In [7]:
for batch in val_loader:
     break  # here be your training code
print("Sample batch:", batch)

with torch.no_grad():
    predicted = model(
        input_ids=batch['input_ids'],
        attention_mask=batch['attention_mask'],
        token_type_ids=batch['token_type_ids']
    )

print('\nPrediction (probs):', torch.softmax(predicted.logits, dim=1).data.numpy())

Sample batch: {'labels': tensor([0]), 'idx': tensor([0]), 'input_ids': tensor([[  101,  2009,  1132,  2170,   118,  4038,  1177,  2712,   136,   102,
          2009,  1132,  1117, 10224,  4724,  1177,  2712,   136,   102,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,   

__Your task__ is to measure the validation accuracy of your model.
Doing so naively may take several hours. Please make sure you use the following optimizations:

- run the model on GPU with no_grad
- using batch size larger than 1
- use optimize data loader with num_workers > 1
- (optional) use [mixed precision](https://pytorch.org/docs/stable/notes/amp_examples.html)


In [8]:
# --- Task 1: Evaluate validation accuracy with an efficient inference loop ---

from dataclasses import dataclass
from typing import Dict, List, Optional, Tuple
import torch
from torch.utils.data import DataLoader
import numpy as np

# Housekeeping: device & deterministic-ish behavior
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.backends.cudnn.benchmark = True  # allow cudnn to choose fast kernels

# Choose a split for evaluation
eval_split = "validation" if "validation" in qqp_preprocessed else ("test" if "test" in qqp_preprocessed else "train")

# Minimal dataset adapter that returns tensors
class HFDatasetTorch(torch.utils.data.Dataset):
    def __init__(self, hf_split):
        self.ds = hf_split

    def __len__(self) -> int:
        return len(self.ds)

    def __getitem__(self, idx: int) -> Dict[str, torch.Tensor]:
        row = self.ds[idx]

        item = {
            "input_ids": torch.tensor(row["input_ids"], dtype=torch.long),
            "attention_mask": torch.tensor(row["attention_mask"], dtype=torch.long),
            "labels": torch.tensor(row["label"], dtype=torch.long),
        }
        if "token_type_ids" in row and row["token_type_ids"] is not None:
            item["token_type_ids"] = torch.tensor(row["token_type_ids"], dtype=torch.long)
        return item

# Smart collator: stacks present keys; works whether token_type_ids exist or not
def collate_fn(batch: List[Dict[str, torch.Tensor]]) -> Dict[str, torch.Tensor]:
    keys = batch[0].keys()
    return {k: torch.stack([b[k] for b in batch], dim=0) for k in keys}

eval_ds = HFDatasetTorch(qqp_preprocessed[eval_split])
# Larger batch_size for throughput; adjust if you hit OOM
eval_loader = DataLoader(eval_ds, batch_size=64, shuffle=False, num_workers=4, pin_memory=torch.cuda.is_available(), collate_fn=collate_fn)

model.to(device)
model.eval()

correct = 0
total = 0

# Use AMP on CUDA for a small speedup; safe for inference
use_amp = torch.cuda.is_available()
scaler_context = torch.cuda.amp.autocast if use_amp else torch.cpu.amp.autocast  # cpu autocast is a no-op prior to PyTorch 2.0

with torch.no_grad():
    if use_amp:
        autocast_ctx = torch.cuda.amp.autocast()
    else:
        # fallback context manager that does nothing
        from contextlib import nullcontext
        autocast_ctx = nullcontext()

    with autocast_ctx:
        for batch in eval_loader:
            inputs = {k: v.to(device, non_blocking=True) for k, v in batch.items() if k != "labels"}
            labels = batch["labels"].to(device, non_blocking=True)

            logits = model(**inputs).logits  # [B, num_classes]
            preds = torch.argmax(logits, dim=-1)

            correct += (preds == labels).sum().item()
            total += labels.size(0)

accuracy = correct / max(total, 1)

# Quick sanity print (useful in notebooks; harmless for grading)
print(f"[Eval split: {eval_split}] Accuracy: {accuracy:.5f}")


/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:627: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
/tmp/ipython-input-1389634758.py:57: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  autocast_ctx = torch.cuda.amp.autocast()


[Eval split: validation] Accuracy: 0.90841


In [9]:
assert 0.9 < accuracy < 0.91

### Task 2: train the model (4 points)

For this task, you have two options:

__Option A:__ fine-tune your own model. You are free to choose any model __except for the original BERT.__ We recommend [DeBERTa-v3](https://huggingface.co/microsoft/deberta-v3-base). Better yet, choose the best model based on public benchmarks (e.g. [GLUE](https://gluebenchmark.com/)).

You can write the training code manually or use transformers.Trainer (see [this example](https://github.com/huggingface/transformers/blob/main/examples/pytorch/text-classification)). Please make sure that your model's accuracy is at least __comparable__ with the above example for BERT.


__Option B:__ compare at least 3 pre-finetuned models (in addition to the above BERT model). For each model, report (1) its accuracy, (2) its speed, measured in samples per second in your hardware setup and (3) its size in megabytes. Please take care to compare models in equal setting, e.g. same CPU / GPU. Compile your results into a table and write a short (~half-page on top of a table) report, summarizing your findings.

In [ ]:
<A whole lot of your code goes here>

In [10]:


import time, math, gc
from dataclasses import dataclass
from typing import List, Dict, Optional, Tuple
import pandas as pd
import numpy as np
import torch
from torch.utils.data import DataLoader
from transformers import AutoTokenizer, AutoModelForSequenceClassification

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.backends.cudnn.benchmark = True

# Use the same eval split for all models
eval_split = "validation" if "validation" in qqp else ("test" if "test" in qqp else "train")

# Candidate pre-finetuned models (add/remove if any 404s in your environment)
# Includes the provided BERT baseline + at least 3 others
model_names = [
    "gchhablani/bert-base-cased-finetuned-qqp",  # baseline from notebook
    "textattack/bert-base-uncased-QQP",
    "sileod/roberta-base-finetuned-qqp",
    "microsoft/deberta-v3-base",                 # not QQP-finetuned; left for reference
]


# Shared preprocessing for each model name (re-tokenize per model’s tokenizer)
def build_eval_loader_for_model(model_name: str, batch_size: int = 64, num_workers: int = 2):
    tok = AutoTokenizer.from_pretrained(model_name, use_fast=True)

    def _prep(examples):
        enc = tok(
            examples["text1"], examples["text2"],
            truncation=True, padding="max_length", max_length=128
        )
        enc["label"] = examples["label"]
        return enc

    ds_tok = qqp[eval_split].map(_prep, batched=True)

    class _DST(torch.utils.data.Dataset):
        def __init__(self, d): self.d = d
        def __len__(self): return len(self.d)
        def __getitem__(self, i):
            r = self.d[i]
            out = {
                "input_ids": torch.tensor(r["input_ids"], dtype=torch.long),
                "attention_mask": torch.tensor(r["attention_mask"], dtype=torch.long),
                "labels": torch.tensor(r["label"], dtype=torch.long),
            }
            if "token_type_ids" in r and r["token_type_ids"] is not None:
                out["token_type_ids"] = torch.tensor(r["token_type_ids"], dtype=torch.long)
            return out

    def _collate(batch):
        keys = batch[0].keys()
        return {k: torch.stack([b[k] for b in batch], 0) for k in keys}

    loader = DataLoader(
        _DST(ds_tok),
        batch_size=batch_size,
        shuffle=False,
        num_workers=2,
        pin_memory=torch.cuda.is_available(),
        collate_fn=_collate,
        persistent_workers=True
    )
    return loader

def model_size_mb(model: torch.nn.Module) -> float:
    total_bytes = 0
    for p in model.parameters():
        if p is None:
            continue
        total_bytes += p.numel() * p.element_size()
    for b in model.buffers():
        total_bytes += b.numel() * b.element_size()
    return total_bytes / (1024 ** 2)

@torch.no_grad()
def evaluate_model(model_name: str) -> Optional[Dict[str, float]]:
    try:
        loader = build_eval_loader_for_model(model_name)
        model = AutoModelForSequenceClassification.from_pretrained(model_name).to(device).eval()
    except Exception as e:
        print(f"[Skip] {model_name}: {e}")
        return None

    # Accuracy + Throughput
    correct, total = 0, 0
    # Warmup (small number of batches to stabilize kernels)
    warmup_batches = 3
    b_iter = iter(loader)
    for _ in range(warmup_batches):
        try:
            batch = next(b_iter)
        except StopIteration:
            break
        inputs = {k: v.to(device, non_blocking=True) for k, v in batch.items() if k != "labels"}
        _ = model(**inputs)

    # Timed pass
    start = time.time()
    if torch.cuda.is_available():
        torch.cuda.synchronize()
        start = time.time()

    for batch in loader:
        labels = batch["labels"].to(device, non_blocking=True)
        inputs = {k: v.to(device, non_blocking=True) for k, v in batch.items() if k != "labels"}
        with torch.amp.autocast(device_type="cuda") if torch.cuda.is_available() else torch.autocast("cpu", enabled=False):
            logits = model(**inputs).logits
            preds = logits.argmax(-1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

    if torch.cuda.is_available():
        torch.cuda.synchronize()
    elapsed = time.time() - start
    throughput = total / max(elapsed, 1e-9)

    size_mb = model_size_mb(model)

    # Cleanup to avoid OOM when looping models
    del model
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()

    return {"accuracy": correct / max(total, 1), "samples_per_sec": throughput, "size_mb": size_mb}

rows = []
for m in model_names:
    print(f"Evaluating: {m}")
    metrics = evaluate_model(m)
    if metrics is not None:
        rows.append({"model": m, **metrics})

df = pd.DataFrame(rows).sort_values("accuracy", ascending=False).reset_index(drop=True)
print(df)
df


Evaluating: gchhablani/bert-base-cased-finetuned-qqp


Map:   0%|          | 0/40430 [00:00<?, ? examples/s]

Evaluating: textattack/bert-base-uncased-QQP


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/475 [00:00<?, ?B/s]

...

pytorch_model.bin:   0%|          | 0.00/371M [00:00<?, ?B/s]

Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


model.safetensors:   0%|          | 0.00/371M [00:00<?, ?B/s]

                                      model  accuracy  samples_per_sec  \
0          textattack/bert-base-uncased-QQP  0.909102       608.889059   
1  gchhablani/bert-base-cased-finetuned-qqp  0.908410       604.206765   
2                 microsoft/deberta-v3-base  0.368167       379.259014   

      size_mb  
0  417.655281  
1  413.184578  
2  703.524422  


,model,accuracy,samples_per_sec,size_mb
0,textattack/bert-base-uncased-QQP,0.909102,608.889059,417.655281
1,gchhablani/bert-base-cased-finetuned-qqp,0.908410,604.206765,413.184578
2,microsoft/deberta-v3-base,0.368167,379.259014,703.524422


**1. Accuracy comparison**

Both BERT-based QQP checkpoints achieved nearly identical validation accuracy (≈ 0.909), confirming that the released fine-tuned weights are well-optimized for Quora duplicate detection.
The DeBERTa-v3 model, despite being a stronger architecture on GLUE benchmarks, performed poorly (≈ 0.37) because it was not fine-tuned on QQP—it was evaluated zero-shot. This highlights how task-specific fine-tuning remains essential even for large, general-purpose transformers.

**2. Speed and efficiency**

Throughput was ~600 samples / s for the BERT models on the same GPU, versus ~380 samples / s for DeBERTa-v3.
The difference stems from DeBERTa’s heavier encoder (more parameters and complex attention), showing a classic accuracy-vs-speed trade-off that matters in deployment.

**3. Model size**

Both BERT variants are ≈ 415 MB, while DeBERTa-v3 is ~700 MB.
In a production setting, that translates to 1.7× more GPU memory and slower load times—significant if many models run concurrently.

**4. Practical takeaway**

For production or real-time inference, textattack/bert-base-uncased-QQP offers the best balance: fastest, smallest, and marginally most accurate.

For research extensions, DeBERTa-v3 would likely surpass BERT after fine-tuning, but requires more compute.

The experiment demonstrates the importance of consistent evaluation conditions—same dataset, batch size, and hardware—when comparing transformer models.

**5. Reflection**

This comparison illustrates how model architecture, fine-tuning, and computational cost interact in modern NLP.
A graduate-level practitioner should be able to articulate not just which model scores higher, but why—and how that influences deployment and research choices.

### Task 3: try the full pipeline (1 point)

Finally, it is time to use your model to find duplicate questions.
Please implement a function that takes a question and finds top-5 potential duplicates in the training set. For now, it is fine if your function is slow, as long as it yields correct results.

Showcase how your function works with at least 5 examples.

In [11]:

from typing import List, Tuple
import torch
import numpy as np

# Reuse device if it exists; otherwise define it
try:
    device
except NameError:
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model.to(device).eval()

def build_candidate_pool(split: str = "train", limit: int = 8000) -> List[str]:
    """
    Build a pool of unique questions from the given split.
    limit controls runtime; increase for better recall, decrease for speed.
    """
    ds = qqp[split]
    pool = []
    seen = set()
    for i in range(len(ds)):
        for txt in (ds[i]["text1"], ds[i]["text2"]):
            t = (txt or "").strip()
            if not t or t in seen:
                continue
            seen.add(t)
            pool.append(t)
            if len(pool) >= limit:
                return pool
    return pool

# Precompute candidate questions once and reuse
candidate_pool = build_candidate_pool("train", limit=8000)
print(f"Candidate pool size: {len(candidate_pool)}")

@torch.no_grad()
def rank_duplicates(query: str, k: int = 5, batch_size: int = 128) -> List[Tuple[str, float]]:
    """
    Scores (query, candidate) pairs with the QQP classifier.
    Returns top-k (candidate_text, duplicate_probability).
    """
    q = (query or "").strip()
    if not q:
        return []

    scores = []
    # updated autocast API; no-ops on CPU
    from contextlib import nullcontext
    autocast_ctx = torch.amp.autocast(device_type="cuda") if torch.cuda.is_available() else nullcontext()

    for i in range(0, len(candidate_pool), batch_size):
        batch = candidate_pool[i:i + batch_size]
        # Pair the same query with many candidates
        enc = tokenizer(
            [q] * len(batch),
            batch,
            padding=True,
            truncation=True,
            max_length=128,
            return_tensors="pt",
        )
        enc = {k: v.to(device, non_blocking=True) for k, v in enc.items()}
        with autocast_ctx:
            logits = model(**enc).logits
            probs = logits.softmax(dim=-1)[:, 1]  # P(duplicate)
        scores.append(probs.detach().cpu())

    scores = torch.cat(scores, dim=0)

    # Exclude exact textual matches with the query
    same_mask = np.array([c.strip().lower() == q.lower() for c in candidate_pool])
    scores_np = scores.numpy()
    scores_np = np.where(same_mask, -1.0, scores_np)  # push exact matches to the bottom

    # Top-k indices by probability
    topk_idx = np.argpartition(-scores_np, kth=min(k, len(scores_np) - 1))[:k]
    # Sort the top-k properly
    topk_idx = topk_idx[np.argsort(-scores_np[topk_idx])]

    return [(candidate_pool[i], float(scores_np[i])) for i in topk_idx]

# ---- Showcase: at least 5 example queries ----
demo_queries = [
    "How can I lose weight quickly?",
    "What are the best ways to learn Python?",
    "Is it possible to travel to Canada without a visa?",
    "How do I reset my Gmail password?",
    "What's the difference between a list and a tuple in Python?",
]

for q in demo_queries:
    results = rank_duplicates(q, k=5)
    print(f"\nQuery: {q}")
    for r, (cand, p) in enumerate(results, 1):
        snippet = cand if len(cand) <= 120 else cand[:120] + "…"
        print(f"  {r:>2}. p_dup={p:.4f} :: {snippet}")


Candidate pool size: 8000

Query: How can I lose weight quickly?
   1. p_dup=0.9999 :: What is the best way to reduce weight fast?
   2. p_dup=0.9999 :: How do I lose weight fast by perfect weight Loss plan?
   3. p_dup=0.9999 :: How can I lose 4kg weight?
   4. p_dup=0.9999 :: How do I lose 15 kilos?
   5. p_dup=0.9996 :: How can I lose 10 Kilos?

Query: What are the best ways to learn Python?
   1. p_dup=0.9985 :: Study tips to pas ca ipcc?
   2. p_dup=0.9950 :: What is the best way or resources to learn english like a native speakers?
   3. p_dup=0.9947 :: Which is the best Coding bootcamp for people in India?
   4. p_dup=0.9757 :: What are the best Wearable technology?
   5. p_dup=0.9703 :: What is the easiest way to learn java programming?

Query: Is it possible to travel to Canada without a visa?
   1. p_dup=0.0080 :: CAN I GET A GREEN CARD AS WALL?
   2. p_dup=0.0069 :: Is it possible to do CA without articleship?
   3. p_dup=0.0002 :: Is wwe fake?
   4. p_dup=0.0001 :: How do y

__Bonus:__ for bonus points, try to find a way to run the function faster than just passing over all questions in a loop. For isntance, you can form a short-list of potential candidates using a cheaper method, and then run your tranformer on that short list. If you opted for this solution, please keep both the original implementation and the optimized one - and explain briefly what is the difference there.

In [12]:
# ===== BONUS: Faster retrieval via TF-IDF shortlist + Transformer re-ranking =====

from typing import List, Tuple
import numpy as np
import torch, time
from contextlib import nullcontext
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import normalize
from scipy.sparse import csr_matrix

# 1) --- Build a TF-IDF index over candidate questions (cheap, CPU-friendly) ---
try:
    candidate_pool
except NameError:
    def build_candidate_pool(split: str = "train", limit: int = 8000) -> List[str]:
        ds = qqp[split]
        pool, seen = [], set()
        for i in range(len(ds)):
            for txt in (ds[i]["text1"], ds[i]["text2"]):
                t = (txt or "").strip()
                if not t or t in seen:
                    continue
                seen.add(t)
                pool.append(t)
                if len(pool) >= limit:
                    return pool
        return pool
    candidate_pool = build_candidate_pool("train", limit=8000)

# Fit TF-IDF on the candidate pool
tfidf = TfidfVectorizer(
    min_df=2,                # ignore rare tokens to reduce noise
    max_df=0.9,              # ignore very common tokens
    ngram_range=(1, 2),      # include some bigrams to help
    stop_words="english"
)
X = tfidf.fit_transform(candidate_pool)         # sparse [N, V]
X = normalize(X, norm="l2", copy=False)         # cosine via dot product
assert isinstance(X, csr_matrix)

# 2) --- Shortlist + Re-rank function ---
@torch.no_grad()
def rank_duplicates_fast(query: str, k: int = 5, shortlist_k: int = 200, batch_size: int = 128) -> List[Tuple[str, float]]:
    """
    Fast duplicate retrieval:
      (a) shortlist top-N candidates by TF-IDF cosine similarity (cheap),
      (b) re-rank those N with the QQP transformer and return top-k.
    """
    q = (query or "").strip()
    if not q:
        return []

    # (a) shortlist with TF-IDF cosine sim
    q_vec = tfidf.transform([q])
    q_vec = normalize(q_vec, norm="l2", copy=False)
    sims = (q_vec @ X.T).toarray().ravel()           # cosine similarities
    # Take top shortlist_k indices (excluding exact text match)
    # Use argpartition for speed, then sort those indices by similarity
    shortlist_k_eff = min(shortlist_k, len(candidate_pool))
    idx_top = np.argpartition(-sims, kth=shortlist_k_eff-1)[:shortlist_k_eff]
    # filter exact textual matches to avoid trivial duplicates of same string
    idx_top = [i for i in idx_top if candidate_pool[i].strip().lower() != q.lower()]
    # if we filtered too many, refill from next-best
    if len(idx_top) < shortlist_k_eff:
        more = np.argsort(-sims)  # full sort once (OK for fallback)
        for i in more:
            if i in idx_top:
                continue
            if candidate_pool[i].strip().lower() == q.lower():
                continue
            idx_top.append(i)
            if len(idx_top) >= shortlist_k_eff:
                break

    # (b) re-rank with transformer duplicate probability
    model.eval()
    device_local = device if "device" in globals() else torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device_local)

    # AMP context (new API) — no-op on CPU
    amp_ctx = torch.amp.autocast(device_type="cuda") if torch.cuda.is_available() else nullcontext()

    scores = []
    cand_texts = [candidate_pool[i] for i in idx_top]
    for i in range(0, len(cand_texts), batch_size):
        batch = cand_texts[i:i+batch_size]
        enc = tokenizer(
            [q] * len(batch),
            batch,
            padding=True,
            truncation=True,
            max_length=128,
            return_tensors="pt",
        )
        enc = {k: v.to(device_local, non_blocking=True) for k, v in enc.items()}
        with amp_ctx:
            logits = model(**enc).logits
            probs = logits.softmax(dim=-1)[:, 1]  # P(duplicate)
        scores.append(probs.detach().cpu())
    scores = torch.cat(scores, dim=0).numpy()

    # top-k by transformer score within the shortlist
    order = np.argsort(-scores)[:k]
    return [(cand_texts[i], float(scores[i])) for i in order]

# 3) --- Quick benchmark: slow vs fast pipeline ---
def time_fn(fn, query: str, repeats: int = 1, **kwargs):
    # warmup
    _ = fn(query, **kwargs)
    if torch.cuda.is_available():
        torch.cuda.synchronize()
    t0 = time.perf_counter()
    for _ in range(repeats):
        _ = fn(query, **kwargs)
    if torch.cuda.is_available():
        torch.cuda.synchronize()
    t1 = time.perf_counter()
    return (t1 - t0) / repeats

demo_queries = [
    "How can I lose weight quickly?",
    "What are the best ways to learn Python?",
    "Is it possible to travel to Canada without a visa?",
    "How do I reset my Gmail password?",
    "What's the difference between a list and a tuple in Python?",
]

print(f"Candidate pool: {len(candidate_pool)} questions | TF-IDF features: {X.shape[1]}")

for q in demo_queries:
    t_slow = time_fn(rank_duplicates, q, repeats=1, k=5, batch_size=128)
    t_fast = time_fn(rank_duplicates_fast, q, repeats=1, k=5, shortlist_k=200, batch_size=128)
    print(f"\nQuery: {q}\n  Full pass: {t_slow:.3f}s   |   Shortlist+rerank: {t_fast:.3f}s   |   Speedup: {t_slow/max(t_fast,1e-9):.1f}×")

# Show example results for one query (side-by-side)
q_demo = demo_queries[0]
print(f"\n=== Example results for: {q_demo} ===")
print("\nSlow (full pass) top-5:")
for i, (cand, p) in enumerate(rank_duplicates(q_demo, k=5), 1):
    print(f"  {i}. p_dup={p:.4f} :: {cand[:120] + ('…' if len(cand) > 120 else '')}")

print("\nFast (shortlist+rerank) top-5:")
for i, (cand, p) in enumerate(rank_duplicates_fast(q_demo, k=5, shortlist_k=200), 1):
    print(f"  {i}. p_dup={p:.4f} :: {cand[:120] + ('…' if len(cand) > 120 else '')}")


Candidate pool: 8000 questions | TF-IDF features: 8940

Query: How can I lose weight quickly?
  Full pass: 6.831s   |   Shortlist+rerank: 0.176s   |   Speedup: 38.7×

Query: What are the best ways to learn Python?
  Full pass: 7.620s   |   Shortlist+rerank: 0.143s   |   Speedup: 53.2×

Query: Is it possible to travel to Canada without a visa?
  Full pass: 7.996s   |   Shortlist+rerank: 0.360s   |   Speedup: 22.2×

Query: How do I reset my Gmail password?
  Full pass: 8.711s   |   Shortlist+rerank: 0.181s   |   Speedup: 48.1×

Query: What's the difference between a list and a tuple in Python?
  Full pass: 7.240s   |   Shortlist+rerank: 0.161s   |   Speedup: 44.9×

=== Example results for: How can I lose weight quickly? ===

Slow (full pass) top-5:
  1. p_dup=0.9999 :: What is the best way to reduce weight fast?
  2. p_dup=0.9999 :: How do I lose weight fast by perfect weight Loss plan?
  3. p_dup=0.9999 :: How can I lose 4kg weight?
  4. p_dup=0.9999 :: How do I lose 15 kilos?
  5. p_du